In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 8.3 MB/s 
     |████████████████████████████████| 408 kB 59.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.7 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.7 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.7 which is incompatible.
google-api-core 1.31.6 requires protobuf<4.0.0dev,>=3.12.0; python_version > "3", but you have proto

In [ ]:
from ortools.linear_solver import pywraplp

In [20]:
import math

In [32]:
demand = [0, 300, 300, 350, 400, 450, 500, 650, 600, 475, 475, 450, 450]
initial_inventory = 50
initial_workers = 40

def LinearProgrammingExample(initial_inventory = initial_inventory, initial_workers = initial_workers, demand = demand):
    """Linear programming sample."""
    
    solver = pywraplp.Solver.CreateSolver('SCIP') # Integer Optimization
    if not solver:
        return

    # Create Variables
    I0 = solver.IntVar(0, solver.infinity(), 'I0') 
    W0 = solver.IntVar(0, solver.infinity(), 'W0') 

    for i in range(1, 13):
      globals()['X' + str(i)] = solver.IntVar(0, solver.infinity(), 'X %s' % (i))
      globals()['I' + str(i)] = solver.IntVar(0, solver.infinity(), 'I %s' % (i))
      globals()['W' + str(i)] = solver.IntVar(0, solver.infinity(), 'W %s' % (i))
      globals()['H' + str(i)] = solver.IntVar(0, solver.infinity(), 'H %s' % (i))
      globals()['F' + str(i)] = solver.IntVar(0, solver.infinity(), 'F %s' % (i))
  
    print('Number of variables =', solver.NumVariables())

    # Workers-quantity relationship
    for i in range(1, 13):
      solver.Add( globals()['X' + str(i)] - 10*globals()['W' + str(i)] == 0 )

    # Inventory flow conservation
    solver.Add( I0 == initial_inventory)
    solver.Add( I1 - I0 - X1 + demand[1] == 0)

    for i in range(2, 13):
      solver.Add( globals()['I' + str(i)] - globals()['I' + str(i-1)] - globals()['X' + str(i)] + demand[i] == 0 )

    # Workers conservation
    solver.Add(W0 == initial_workers)
    solver.Add( W1 - W0 - H1 + F1 == 0 )

    for i in range(2, 13):
     solver.Add( globals()['W' + str(i)] - globals()['W' + str(i-1)] - globals()['H' + str(i)] + globals()['F' + str(i)] == 0 )

    # Constraints
    for i in range(1, 13):
      solver.Add( globals()['X' + str(i)] - 10 * globals()['W' + str(i)] == 0)
      # solver.Add( globals()['I' + str(i)] - globals()['I' + str(i-1)] - globals()['X' + str(i)] + demand[i] == 0)
      # solver.Add( globals()['W' + str(i)] - globals()['W' + str(i-1)] - globals()['H' + str(i)] + globals()['F' + str(i)] == 0 )

    print('Number of constraints =', solver.NumConstraints())

    # Objective function
    sum_W = 0
    for i in range(1, 13):
       sum_W += globals()['W' + str(i)]
    
    sum_H = 0
    for i in range(1, 13):
       sum_H += globals()['H' + str(i)]

    sum_F = 0
    for i in range(1, 13):
       sum_F += globals()['F' + str(i)]

    sum_I = 0
    for i in range(1, 13):
       sum_I += globals()['I' + str(i)]

    solver.Minimize(2*sum_W + sum_H + 2*sum_F + 0.06*sum_I)

    # Solve the system.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print('-'*30)
        for i in range(1, 13):
          print(f'X{i} =', globals()['X' + str(i)].solution_value())
          print(f'I{i} =', globals()['I' + str(i)].solution_value())
          print(f'W{i} =', globals()['W' + str(i)].solution_value())
          print(f'H{i} =', globals()['H' + str(i)].solution_value())
          print(f'F{i} =', globals()['F' + str(i)].solution_value())
          print('-'*30)
        
    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())


LinearProgrammingExample(initial_inventory, initial_workers, demand)

Number of variables = 62
Number of constraints = 50
Solution:
Objective value = 1157.5
------------------------------
X1 = 300.0
I1 = 50.0
W1 = 30.0
H1 = 0.0
F1 = 10.0
------------------------------
X2 = 300.0
I2 = 50.0
W2 = 30.0
H2 = 0.0
F2 = 0.0
------------------------------
X3 = 300.0
I3 = 0.0
W3 = 30.0
H3 = 0.0
F3 = 0.0
------------------------------
X4 = 400.0
I4 = 0.0
W4 = 40.0
H4 = 10.0
F4 = 0.0
------------------------------
X5 = 460.0
I5 = 10.0
W5 = 46.0
H5 = 6.0
F5 = 0.0
------------------------------
X6 = 580.0
I6 = 90.0
W6 = 58.0
H6 = 12.0
F6 = 0.0
------------------------------
X7 = 580.0
I7 = 20.0
W7 = 58.0
H7 = 0.0
F7 = 0.0
------------------------------
X8 = 580.0
I8 = 0.0
W8 = 58.0
H8 = 0.0
F8 = 0.0
------------------------------
X9 = 480.0
I9 = 5.0
W9 = 48.0
H9 = 0.0
F9 = 10.0
------------------------------
X10 = 470.0
I10 = 0.0
W10 = 47.0
H10 = 0.0
F10 = 1.0
------------------------------
X11 = 450.0
I11 = 0.0
W11 = 45.0
H11 = 0.0
F11 = 2.0
-------------------------

In [34]:
LinearProgrammingExample(12, 15, demand)

Number of variables = 62
Number of constraints = 50
Solution:
Objective value = 1155.9400000000003
------------------------------
X1 = 290.0
I1 = 2.0
W1 = 29.0
H1 = 14.0
F1 = -0.0
------------------------------
X2 = 300.0
I2 = 2.0
W2 = 30.0
H2 = 1.0
F2 = 0.0
------------------------------
X3 = 350.0
I3 = 2.0
W3 = 35.0
H3 = 5.0
F3 = 0.0
------------------------------
X4 = 400.0
I4 = 2.0
W4 = 40.0
H4 = 5.0
F4 = 0.0
------------------------------
X5 = 460.0
I5 = 12.0
W5 = 46.0
H5 = 6.0
F5 = 0.0
------------------------------
X6 = 580.0
I6 = 92.0
W6 = 58.0
H6 = 12.0
F6 = 0.0
------------------------------
X7 = 580.0
I7 = 22.0
W7 = 58.0
H7 = 0.0
F7 = 0.0
------------------------------
X8 = 580.0
I8 = 2.0
W8 = 58.0
H8 = 0.0
F8 = -0.0
------------------------------
X9 = 480.0
I9 = 7.0
W9 = 48.0
H9 = 0.0
F9 = 10.0
------------------------------
X10 = 470.0
I10 = 2.0
W10 = 47.0
H10 = 0.0
F10 = 1.0
------------------------------
X11 = 450.0
I11 = 2.0
W11 = 45.0
H11 = 0.0
F11 = 2.0
--------------